In [8]:
import pandas as pd
import numpy as np
from concurrent.futures import ProcessPoolExecutor
from sklearn.linear_model import LinearRegression
import time
import requests
import asyncio
import aiohttp

data = pd.read_csv('temperature_data.csv')
data['timestamp'] = pd.to_datetime(data['timestamp'])
data = data.sort_values(by='timestamp')


def get_season_name(season_number):
    seasons = {
        1: 'Зима',
        2: 'Весна',
        3: 'Лето',
        4: 'Осень'
    }
    return seasons.get(season_number, 'Неизвестный сезон')

# Функция для анализа данных по одному городу
def analyze_city(city_data):

    city_data = city_data.copy()

    city_data.loc[:, 'season'] = city_data['timestamp'].dt.month % 12 // 3 + 1

    # Вычисляем скользящее среднее и стандартное отклонение
    city_data.loc[:, 'rolling_mean'] = city_data['temperature'].rolling(window=30).mean()
    city_data.loc[:, 'rolling_std'] = city_data['temperature'].rolling(window=30).std()

    # Выявление аномалий
    anomalies = city_data[(city_data['temperature'] > city_data['rolling_mean'] + 2 * city_data['rolling_std']) |
                          (city_data['temperature'] < city_data['rolling_mean'] - 2 * city_data['rolling_std'])]

    # Группировка по сезонам и расчет mean и std
    season_profile = city_data.groupby('season')['temperature'].agg(['mean', 'std']).reset_index()

    # Тренд
    X = np.arange(len(city_data)).reshape(-1, 1)
    y = city_data['temperature'].values
    model = LinearRegression().fit(X, y)
    trend = model.coef_[0]

    # Средняя, минимальная и максимальная температура
    average_temp = city_data['temperature'].mean()
    min_temp = city_data['temperature'].min()
    max_temp = city_data['temperature'].max()

    return {
        'city': city_data['city'].iloc[0],
        'average_temp': average_temp,
        'min_temp': min_temp,
        'max_temp': max_temp,
        'season_profile': season_profile,
        'trend': trend,
        'anomalies': anomalies
    }

# Получение текущей температуры (синхронно)
def get_current_temperature(city, api_key):
    response = requests.get(f"http://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}")
    if response.status_code == 200:
        current_weather = response.json()
        current_temp = current_weather['main']['temp'] - 273.15
        return current_temp
    else:
        raise Exception("Ошибка получения данных: " + response.json().get("message", "Неизвестная ошибка"))

# Получение текущей температуры (асинхронно)
async def get_current_temperature_async(city, api_key):
    async with aiohttp.ClientSession() as session:
        async with session.get(f"http://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}") as response:
            if response.status == 200:
                current_weather = await response.json()
                current_temp = current_weather['main']['temp'] - 273.15
                return current_temp
            else:
                raise Exception("Ошибка получения данных: " + (await response.json()).get("message", "Неизвестная ошибка"))

# Параллелизация анализа
def parallel_analysis(data):
    cities = data['city'].unique()
    city_data_list = [data[data['city'] == city] for city in cities]

    start_time = time.time()
    with ProcessPoolExecutor() as executor:
        results = list(executor.map(analyze_city, city_data_list))
    parallel_time = time.time() - start_time

    return results, parallel_time

# Анализ без параллелизации
def sequential_analysis(data):
    cities = data['city'].unique()
    results = []

    start_time = time.time()
    for city in cities:
        city_data = data[data['city'] == city]
        result = analyze_city(city_data)
        results.append(result)
    sequential_time = time.time() - start_time

    return results, sequential_time

# Основная функция
def main_analysis(api_key, city_name):
    # Фильтруем данные для выбранного города
    city_data = data[data['city'] == city_name]

    # Выполнение анализа
    parallel_results, parallel_time = parallel_analysis(city_data)
    sequential_results, sequential_time = sequential_analysis(city_data)

    try:
        current_temp = get_current_temperature(city_name, api_key)
        print(f"Текущая температура в {city_name}: {current_temp:.2f} °C")

        # Определение нормальности текущей температуры
        city_data.loc[:, 'season'] = city_data['timestamp'].dt.month % 12 // 3 + 1
        current_season = city_data['season'].iloc[-1]
        season_name = get_season_name(current_season)
        season_profile = city_data.groupby('season')['temperature'].agg(['mean', 'std']).loc[current_season]

        if (current_temp > season_profile['mean'] - 2 * season_profile['std']) and (current_temp < season_profile['mean'] + 2 * season_profile['std']):
            print(f"Температура в {city_name} нормальная для текущего сезона: {season_name}.")
        else:
            print(f"Температура в {city_name} аномальная для текущего сезона: {season_name}.")
    except Exception as e:
        print(f"Ошибка при получении температуры для {city_name}: {e}")

    print(f"Время выполнения с параллелизацией: {parallel_time:.2f} секунд")
    print(f"Время выполнения без параллелизации: {sequential_time:.2f} секунд")

if __name__ == "__main__":
    api_key = "3d3c80fe9fc6fabc5c6777c6e1ece035"
    city_name = "Moscow"
    start_time = time.time()
    main_analysis(api_key, city_name)
    print(f"Общее время выполнения: {time.time() - start_time} секунд")


Текущая температура в Moscow: 0.37 °C
Температура в Moscow нормальная для текущего сезона: Зима.
Время выполнения с параллелизацией: 0.09 секунд
Время выполнения без параллелизации: 0.02 секунд
Общее время выполнения: 0.19191527366638184 секунд


In [10]:
if __name__ == "__main__":
    api_key = "3d3c80fe9fc6fabc5c6777c6e1ece035"
    city_name = "Berlin"
    start_time = time.time()
    main_analysis(api_key, city_name)
    print(f"Общее время выполнения: {time.time() - start_time} секунд")


Текущая температура в Berlin: 1.41 °C
Температура в Berlin нормальная для текущего сезона: Зима.
Время выполнения с параллелизацией: 0.08 секунд
Время выполнения без параллелизации: 0.01 секунд
Общее время выполнения: 0.1756885051727295 секунд


In [11]:
if __name__ == "__main__":
    api_key = "3d3c80fe9fc6fabc5c6777c6e1ece035"
    city_name = "Sydney"
    start_time = time.time()
    main_analysis(api_key, city_name)
    print(f"Общее время выполнения: {time.time() - start_time} секунд")

Текущая температура в Sydney: 21.49 °C
Температура в Sydney нормальная для текущего сезона: Зима.
Время выполнения с параллелизацией: 0.08 секунд
Время выполнения без параллелизации: 0.01 секунд
Общее время выполнения: 0.1742873191833496 секунд


In [12]:
data.head()

,city,timestamp,temperature,season
0,New York,2010-01-01,3.612133,winter
10950,Tokyo,2010-01-01,8.880101,winter
40150,Singapore,2010-01-01,29.587655,winter
51100,Mexico City,2010-01-01,14.379541,winter
32850,Dubai,2010-01-01,14.331205,winter
